In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import cv2
import os
import numpy as np
from google.colab.patches import cv2_imshow

In [7]:
def calculatef1(path_folder):
  gt_file = [file for file in os.listdir(path_folder) if file.endswith("_gt.txt")][0]
  frame_file = [file for file in os.listdir(path_folder) if file.endswith("_frames.txt")][0]
  with open(os.path.join(path_folder, frame_file), 'r') as file:
        frame_line = file.read()
  frame_list = [int(x) for x in frame_line.split(",")]
  with open(os.path.join(path_folder, gt_file), 'r') as file:
        lines = file.readlines()
  picture_paths =[]
  for picture in os.listdir(os.path.join(path_folder,"img")):
    if not picture.endswith(".jpg"):
      continue
    if frame_list[0]<=int(picture.split(".")[0])<=frame_list[1]:
      picture_paths.append(os.path.join(path_folder,"img",picture))
  frame = cv2.imread(picture_paths[0])
  prev_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  prev_pts = cv2.goodFeaturesToTrack(prev_gray, maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)

  # Create a mask image for drawing purposes
  mask = np.zeros_like(frame)

  # Initialize variables for IoU calculation
  true_positive = 0
  false_positive = 0
  false_negative = 0
  i=0
  bbox = [int(x) for x in lines[0].split(",")]
  for picture in picture_paths:
    frame = cv2.imread(picture)
    # Convert the current frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Calculate optical flow
    curr_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev_pts, None)
    
    # Select good points for tracking
    if curr_pts is not None and len(curr_pts) > 0:
            good_new = curr_pts[status == 1]
            good_old = prev_pts[status == 1]

            # Estimate the motion vector
            if len(good_new) > 0 and len(good_old) > 0:
                # Estimate the motion vector
                motion_vector = np.mean(good_new - good_old, axis=0)

                # Update the bounding box position based on the motion vector
                x, y, w, h = bbox
                motion_vector = motion_vector.astype(int)
                scaling_factor = 0.1
                motion_vector_scaled = [v * scaling_factor for v in motion_vector]
                bbox = (int(x + motion_vector_scaled[0]), int(y + motion_vector_scaled[1]), w, h)
           
    else:
            pass      
    # Compute ground truth bounding box for the current frame
    # (You can manually specify or use an object detection algorithm)
    # gt_bbox = [xmin, ymin, xmax, ymax]
    gt_bbox = [int(float(x)) for x in lines[i].split(",")]
    # Compute predicted bounding box based on the tracked points
    # pred_bbox = [xmin, ymin, xmax, ymax]
    pred_bbox = bbox
    # Compute IoU
    intersection_area = max(0, abs(min(pred_bbox[2], gt_bbox[2]) - max(pred_bbox[0], gt_bbox[0]))) * \
                           max(0, abs(min(pred_bbox[3], gt_bbox[3]) - max(pred_bbox[1], gt_bbox[1])))
    union_area = abs(pred_bbox[2] - pred_bbox[0]) * abs(pred_bbox[3] - pred_bbox[1]) + \
                     abs(gt_bbox[2] - gt_bbox[0]) * abs(gt_bbox[3] - gt_bbox[1]) -intersection_area

    if union_area != 0:
            iou = intersection_area / union_area
    else:
            iou = 0
    # Update true positive, false positive, and false negative counts
    if iou >= 0.5:
        true_positive += 1
    else:
        false_positive += 1
        false_negative += 1
           
    # Update previous frame and points
    prev_gray = gray.copy()
    prev_pts = good_new.reshape(-1, 1, 2)
    i=i+1
  precision = true_positive / (true_positive + false_positive)
  recall = true_positive / (true_positive + false_negative)
  f1_score = 2 * (precision * recall) / (precision + recall)
  return f1_score


In [8]:
folder_path = "/content/drive/MyDrive/CV"
folder_paths = []
for folder in os.listdir(folder_path):
    folder_paths.append(os.path.join(folder_path, folder,folder))
accuracy = 0
for files in folder_paths:
  accuracy+=calculatef1(files) 


In [9]:
accuracy /=len(folder_paths)

In [10]:
print(accuracy)

0.7655691398814122
